## 0 - Demographics

a. Identify IDs with 3T and 7T  
b. Extract clinical information for epilepsy patients  
c. Extract demographic information for all participants  

In [35]:
import pandas as pd
import os
import sys
import importlib

sys.path.append("/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/code")
from Utils import id, gen, t1

## 1. Identify IDs

In [ ]:
src_dir = "/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/projects/PT/sources" # path to directory with source pt sheets

# For each sheet, must define NAME, PATH, SHEET, ID_7T, ID_3T. 
# All other keys are those to be extracted.
# The same variables should have the same key names across sheets.

PNI = {
    'NAME': 'PNI',
    'PATH': f'{src_dir}/MICA_PNI_19May2025.xlsx', # 7T controls
    'SHEET': 'all', # name of sheet in file
    'ID_7T': 'ID_PNI', 
    'ID_3T': 'ID_MICs',
    'Ses_7T': 'session',
    'Date_7T': 'scanDate',
    'study': '7T',
    'Sex': 'sex',
    'Gender': 'gender',
    'Hand': 'handedness',
    'Eth': 'ethnicity',
    'Job': 'employment',
    'Edu': 'education',
    'LastSz': 'lastSeizure'
}

MICs = {
    'NAME': 'MICs',
    'PATH': f'{src_dir}/MICA-MTL-3T_19May2025.xlsx', # 3T controls
    'SHEET': 'Sheet1', # name of sheet in file
    'ID_7T': None, 
    'ID_3T': 'Study_name',
    'Ses_3T': 'Visit',
    'Date_3T': 'Scan_Date (D.M.Y)',
    'study': '3T',
    'Hand': 'Handed', 
    'Sex': 'AssignedSex',
    'Gender': 'Gender',
    'Height': 'HeightApprox',
    'Weight': 'WeightApprox',
    'Eth': 'Ethnicity',
    'Glasses': 'Glasses',
    'Job': 'Employ',
    'Edu': 'YoE',
    'LastSz': 'Last seizure'
}

Clin = {
    'NAME': 'Clin',
    'PATH': f'{src_dir}/Clinical_19May2025.xlsx',
    'SHEET': 'Sheet1', # name of sheet in file
    'ID_7T': None, 
    'ID_3T': 'participant_id',
    'scanDate_3t': 'scan_date',
    'Gender': 'Gender',
    'Hand': 'Handedness',
    'Language': 'Language',
    'Job': 'Employment',
    'Edu': 'Education',
    'EpilepsyDxILAE': 'Epilepsy diagnosis based on ILAE',
    'EpilepsyClass': 'Epilepsy classification:Focal,Generalized',
    'FocusLat': 'Lateralization of epileptogenic focus',
    'FocusConfirmed': 'Epileptogenic focus confirmed by the information of (sEEG/ site of surgical resection/ Ictal EEG abnormalities +/. MRI findings): FLE=forntal lobe epilepsy and cingulate epilepsy, CLE:central/midline epilepsy,ILE: insular epilepsy, mTLE=mesio.temporal lobe epilepsy, nTLE=neocortical lobe epilepsy, PQLE=posterior quadrant lobe epilepsy , multifocal epilepsy,IGE=ideopathic lobe epilepsy,unclear)',
    'EMUDischargeDx': 'Dx at EMU discharge ',
    'EMUAdmissionDate': 'EMU admission date(dd-mm-yy)',
    'AdmissionDuration': 'Duration of admission',
    'EpilepsyRiskFactors': 'Risk factors for epilepsy',
    'SeizureOnsetYr': 'Seizure onset (yr)',
    'DrugResistant': 'Drug resistant epilepsy at time of EMU admission',
    'NumASMsPrior': '# of ASMs prior current EMU admission',
    'PrevASMs': 'Previous ASMs (name and doses (mg/d)) if applicable prior the current EMU admission',
    'NumASMOnAdmission': '# of ASM on admission',
    'ASMsOnAdmission': 'ASMs  on admission (name, doses (mg per day)',
    'GeneticTest': 'Genetic test (year,results)',
    'FDGPET': 'FDG.PET',
    'BaselineMRI': 'Baseline MRI (year,results)',
    'InvasiveExplorations': 'Invasive explorations (Y/N)',
    'NumSurgicalResections': '# of surgical resection/thermocoagulatin',
    'SurgicalResectionDateSite': 'Surgical resection date and site',
    'Histopathology': 'Histopatholgy',
    'Engel6mo': 'Engel classification (seizure outcomes at the 6 month )',
    'Engel1yr': 'Engel classification (seizure outcomes after 1 year from surgical resection)',
    'ILAEOutcome1yr': 'ILAE outcome after surgical resection by 1 yr',
    'NeuromodDevices': 'Neuromodulation devices'
    }

DOB_PNI = {
        'NAME': 'DOB_7T',
        'PATH': f'{src_dir}/PNI_DOB_12May.xlsx',
        'SHEET': 'Sheet1', # name of sheet in file
        'ID_7T': 'PNI', 
        'ID_3T': 'MICS',
        'DOB_colName': 'DOB_ddmmyyyy'
        }

sheets = [PNI, MICs, DOB_PNI, Clin]
correspSheets = [PNI]

In [8]:
importlib.reload(id)
ids_out = "/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/projects/3T7T/outputs/pt"
ids = id.ids3T7T(sheets)
ids

[ID_3T7T] Loading: PNI
[ID_3T7T] Skipping: MICs (missing ID_7T or ID_3T key)
[ID_3T7T] Skipping: Clin (missing ID_7T or ID_3T key)
[ID_3T7T] Loading: DOB_7T


,PNI_ID,MICS_ID
0,Pilot013,HC129
1,PNC003,HC082
2,PNC006,HC081
3,PNC011,HC083
4,PNC018,HC076
5,PNC019,HC062
6,PNC024,HC088
7,PNC025,HC128
8,PNC026,HC130
9,PNC031,HC154


## 2. Extract Demographics

In [ ]:
# Extract demographics and all visits for pts identified in the `ids` object
# output should have columns:
    ## MICS_ID, PNI_ID, Study (MICs or PNI), ScanDate, Session, DEMOGRAPHIC VARIABLES

# two types of sheets:
## Those with visit information : will have a column with session and scan date
## Those with demographics only : will not have session variable, will have many other demographic variables defined

# Extract all variables defined in the sheet dictionary (except if their value is '' or None)


In [11]:
# 1. Extract visits
importlib.reload(id)
save_dir = '/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/projects/3T7T/data'
visits = id.id_visits(sheets, ids, save_pth=save_dir, save_name='paired_visits')
visits

[id_visits] Extracting visits for IDs in list
	Skipping: Clin (missing Ses_7T or Ses_3T key)
	Skipping: DOB_7T (missing Ses_7T or Ses_3T key)


	PNI
		Extracting cols: ['ID_MICs', 'ID_PNI', 'scanDate', 'session']
	MICs
		Extracting cols: ['Study_name', 'Scan_Date (D.M.Y)', 'Visit']
		Finding corresponding ID_7T
[id_visits] Saved: /Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/projects/3T7T/data/paired_visits_19May2025.csv


,MICS_ID,PNI_ID,study,SES,Date
0,HC129,Pilot013,7T,05,18.04.2024
1,HC082,PNC003,7T,01,06.05.2022
2,HC082,PNC003,7T,02,13.06.2022
3,HC082,PNC003,7T,03,13.03.2023
4,HC082,PNC003,7T,04,24.10.2023
...,...,...,...,...,...
107,HC130,PNC026,3T,02,15.01.2025
108,HC083,PNC011,3T,02,28.01.2025
109,PX215,PNE020,3T,01,20.03.2025
110,PX216,PNE021,3T,01,26.03.2025


In [64]:
# 2. Add demographics to the df created above
importlib.reload(t1)
save_dir = '/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/projects/3T7T/data'
demo = t1.demo(sheets, visits, save_pth = save_dir)
demo

[demo] Retriving demographics data.
	Overlapping: ['Sex', 'Gender', 'Hand', 'Eth', 'Job', 'Edu', 'LastSz']
	Extracting PNI
		merge_keys: ['ID_7T', 'ID_3T', 'Ses_7T']
	Extracting MICs
		merge_keys: ['ID_3T', 'Ses_3T']
	Extracting Clin
		merge_keys: ['ID_3T']
	Extracting DOB_7T
		merge_keys: ['ID_7T', 'ID_3T']
[id_visits] Saved: /Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/projects/3T7T/data/demo_19May2025.csv


,MICS_ID,PNI_ID,study,SES,Date,sex,handedness,employment,lastSeizure,ethnicity,...,# of surgical resection/thermocoagulatin,Invasive explorations (Y/N),Drug resistant epilepsy at time of EMU admission,Duration of admission,Gender,EMU admission date(dd-mm-yy),Engel classification (seizure outcomes after 1 year from surgical resection),Lateralization of epileptogenic focus,Histopatholgy,DOB_ddmmyyyy
0,HC129,Pilot013,7T,05,18.04.2024,F,L,Full time student,NaN,Canadian,...,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN,NaN
1,HC082,PNC003,7T,01,06.05.2022,F,R,Full time student,NaN,Japanese,...,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN,17.09.1997
2,HC082,PNC003,7T,02,13.06.2022,F,R,Full time student,NaN,Japanese,...,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN,17.09.1997
3,HC082,PNC003,7T,03,13.03.2023,F,R,Full time student,NaN,Japanese,...,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN,17.09.1997
4,HC082,PNC003,7T,04,24.10.2023,F,R,Full time student,NaN,Japanese,...,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN,17.09.1997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,HC130,PNC026,3T,02,15.01.2025,F,R,Full time student,NaN,Chinese,...,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN,30.06.1996
117,HC083,PNC011,3T,02,28.01.2025,M,R,NaN,NaN,Latino,...,NaN,NaN,NaN,NaN,M,NaN,NaN,NaN,NaN,12.05.1996
118,PX215,PNE020,3T,01,20.03.2025,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Y,9 days,F,05.02.2025,NaN,unclear,NaN,03.02.1967
119,PX216,PNE021,3T,01,26.03.2025,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Y,15 days,F,16.01.2025,NaN,right,NaN,06.07.1975
